# Demo - Automated Trading App

![alt text](https://i.imgur.com/eYJJla1.png)


### Tutorial Outline
1. Prerequisite videos
2. Problems with Tensorflow 1.0
3. Tensorflow 2.0 Features
4. Build a stock prediction model w/ Tensorflow 2.0
5. Tensorflow Serving Explained
6. Simple Tensorflow Serving Demo
7. Add User signup/login functionality
8. Add payments functionality
9. Deploying the app to Heroku
10. Ways of Improving the App

# 1 - Prerequisites for this video 

## 4 Steps

### First, watch 

[![IMAGE ALT TEXT HERE](http://img.youtube.com/vi/HhqhFbwiaig/0.jpg)](http://www.youtube.com/watch?v=HhqhFbwiaig)

### Then, watch 

[![IMAGE ALT TEXT HERE](http://img.youtube.com/vi/mrRfpiAwad0/0.jpg)](http://www.youtube.com/watch?v=mrRfpiAwad0)

### Then, watch 

[![IMAGE ALT TEXT HERE](http://img.youtube.com/vi/NzmoPqte4V4/0.jpg)](http://www.youtube.com/watch?v=NzmoPqte4V4)

### Then, watch this playlist

[![IMAGE ALT TEXT HERE](http://img.youtube.com/vi/2FmcHiLCwTU/0.jpg)](http://www.youtube.com/watch?v=2FmcHiLCwTU)

# 2 - Problems with Tensorflow 1.0

## Problem # 1 - Static Computation Graphs

#### Its called 'DataFlow'

![alt text](https://www.researchgate.net/profile/Margaret_Burnett/publication/220391588/figure/fig2/AS:670720233308170@1536923573575/Dataflow-programming-in-Prograph-Here-the-programmer-is-using-the-low-level-primitive.png
)
- Its modeled after a common programming paradigm called Dataflow
- In a dataflow graph, the nodes represent units of computation, and the edges represent the data consumed or produced by a computation.
- For example, in a TensorFlow graph, the tf.matmul operation would correspond to a single node with two incoming edges (the matrices to be multiplied) and one outgoing edge (the result of the multiplication).

#### Why Use DataFlow?

1. Parallelism - easier to execute operations in parallel
2. Distributed Execution - easier to partition the graph
3. Compilation - XLA Compiler generates faster code using a graph structure
4. Portability - Language independnet graph. 

#### Tensorflow 1.0 Steps

1. Import the data, normalize it, or create data input pipeline.
2. Define an algorithm — Define variables, structure of the algo, loss function, optimization technique, etc. Tensorflow creates static computational graphs for this.
3. Feed the data through this computation graph, compute loss from loss function and and update the weights (variables) by backpropagating the error.
4. Stop when you reach some stopping criteria.

![alt text](https://cdn-images-1.medium.com/max/1600/0*uvXAUUtje1B01o_s.png)

In [0]:
from __future__ import print_function

import tensorflow as tf

# Basic constant operations
# The value returned by the constructor represents the output
# of the Constant op.
a = tf.constant(2)
b = tf.constant(3)
c = a + b
d = a * b

# Launch the default graph.
with tf.Session() as sess:
    print("a=2, b=3")
    print("Addition with constants: %i" % sess.run(c))
    print("Multiplication with constants: %i" % sess.run(d))

a=2, b=3
Addition with constants: 5
Multiplication with constants: 6


## Problem #2 - Verbosity (high learning curve)

- Variables, placeholders, servables, tensorboard, sessions, computation graphs, hyperparameter values,  formatting conventions, SO MUCH TO LEARN
- And we haven't even begun to talk about machine learning theory

In [0]:
 ## An example of verbosity!! Randomly taken from https://github.com/carpedm20/DCGAN-tensorflow/blob/master/model.py 
 ## Yo, WTF is going on LOL (i mean tbh i understand, but still its too messy)
  
  def sigmoid_cross_entropy_with_logits(x, y):
      try:
        return tf.nn.sigmoid_cross_entropy_with_logits(logits=x, labels=y)
      except:
        return tf.nn.sigmoid_cross_entropy_with_logits(logits=x, targets=y)

    self.d_loss_real = tf.reduce_mean(
      sigmoid_cross_entropy_with_logits(self.D_logits, tf.ones_like(self.D)))
    self.d_loss_fake = tf.reduce_mean(
      sigmoid_cross_entropy_with_logits(self.D_logits_, tf.zeros_like(self.D_)))
    self.g_loss = tf.reduce_mean(
      sigmoid_cross_entropy_with_logits(self.D_logits_, tf.ones_like(self.D_)))

    self.d_loss_real_sum = scalar_summary("d_loss_real", self.d_loss_real)
    self.d_loss_fake_sum = scalar_summary("d_loss_fake", self.d_loss_fake)
                          
    self.d_loss = self.d_loss_real + self.d_loss_fake

    self.g_loss_sum = scalar_summary("g_loss", self.g_loss)
    self.d_loss_sum = scalar_summary("d_loss", self.d_loss)

    t_vars = tf.trainable_variables()

    self.d_vars = [var for var in t_vars if 'd_' in var.name]
    self.g_vars = [var for var in t_vars if 'g_' in var.name]

    self.saver = tf.train.Saver()

  def train(self, config):
    d_optim = tf.train.AdamOptimizer(config.learning_rate, beta1=config.beta1) \
              .minimize(self.d_loss, var_list=self.d_vars)
    g_optim = tf.train.AdamOptimizer(config.learning_rate, beta1=config.beta1) \
              .minimize(self.g_loss, var_list=self.g_vars)
    try:
      tf.global_variables_initializer().run()
    except:
      tf.initialize_all_variables().run()

    self.g_sum = merge_summary([self.z_sum, self.d__sum,
      self.G_sum, self.d_loss_fake_sum, self.g_loss_sum])
    self.d_sum = merge_summary(
        [self.z_sum, self.d_sum, self.d_loss_real_sum, self.d_loss_sum])
    self.writer = SummaryWriter("./logs", self.sess.graph)

## Problem #3 - Messy APIs

- So many new packages being added
- Lots of deprecated APIs
- Lots of renaming of existing APIs 

![alt text](https://i.imgur.com/4VTU9RB.png)

![alt text](https://i.imgflip.com/2xvf09.jpg)

# Problem #4 - Hard to Debug 




In [0]:
import tensorflow as tf

x = tf.constant(0.)
y = tf.log(x)  + 1. / x ## This line is the problem, but the output doesn't tell us that
z = y + 1.

with tf.Session() as sess: 
  print(z.eval())

nan


## 3 Tensorflow 2.0 Features

### Feature #1 - Rapid prototyping (Eager Execution by Default)

- Eager execution is an imperative programming paradigm that evaluates operations immediately, without building graphs!
- Operations return concrete values instead of constructing a computational graph to run later.
- This makes it easy to get started with TensorFlow and debug models, and it reduces boilerplate as well. 
- This allows for an intuitive interface—Structure your code naturally and use Python data structures. Quickly iterate on small models and small data.
- No sessions or placeholders, instead pass data into functions as an argument

![alt text](https://i.imgur.com/YUlhihi.png)


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf

tf.executing_eagerly() 

x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))

a = tf.constant([[1, 2],
                 [3, 4]])
print(a)

# Use NumPy values
import numpy as np

c = np.multiply(a, b)
print(c)

    100% |████████████████████████████████| 79.9MB 343kB/s 
    100% |████████████████████████████████| 61kB 21.2MB/s 
    100% |████████████████████████████████| 3.0MB 11.4MB/s 
    100% |████████████████████████████████| 419kB 19.4MB/s 
hello, Tensor("MatMul:0", shape=(1, 1), dtype=float32)
Tensor("Const_7:0", shape=(2, 2), dtype=int32)
Tensor("mul_3:0", shape=(2, 2), dtype=int32)


### Feature # 2 - Easier Debugging (Imperative programming style)

- Call ops directly to inspect running models and test changes
- Use standard Python debugging tools for immediate error reporting.


In [0]:
import tensorflow as tf

#yeahhhhh debugging with print, boss status
x = [[2.]]
m = tf.matmul(x, x)
print(m)
z = tf.matmul(m, m)
print(z)
# The 1x1 matrix [[4.]]

Tensor("MatMul_4:0", shape=(1, 1), dtype=float32)
Tensor("MatMul_5:0", shape=(1, 1), dtype=float32)


## Feature #3 - Less verbose (tf.keras)

- tf.keras is now the official high level API
- Distributed training is simple (1 line of code to enable)
- Deprecated APIs have been removed

In [0]:
# This code downloads data and trains a model in like 45 seconds. Sick. 

#import TF
import tensorflow as tf

#enable distributed training, we'll later need to put wrap
# our model + training code with 'with mirrored_strategy_scope()'
# See https://www.tensorflow.org/guide/distribute_strategy
mirrored_strategy = tf.distribute.MirroredStrategy()

#Import handwritten character labeled dataset.
mnist = tf.keras.datasets.mnist

#split it into training and testing data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# build a model using the keras sequential API. So pretty <3 
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

# compile it with the adam (gradient descent) optimizer, pick a loss function
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#Train it!
model.fit(x_train, y_train, epochs=5)

#Test it!
model.evaluate(x_test, y_test)

11493376/11490434 [==============================] - 0s 0us/step
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch 1/5
60000/60000 [==============================] - 10s 162us/sample - loss: 0.2925 - acc: 0.9159
Epoch 2/5
60000/60000 [==============================] - 10s 160us/sample - loss: 0.1408 - acc: 0.9582
Epoch 3/5
60000/60000 [==============================] - 9s 156us/sample - loss: 0.1053 - acc: 0.9682
Epoch 4/5
60000/60000 [==============================] - 9s 156us/sample - loss: 0.0874 - acc: 0.9728
Epoch 5/5
10000/10000 [==============================] - 1s 53us/sample - loss: 0.0757 - acc: 0.9769


[0.07565579186398536, 0.9769]

## Feature #4 -  More granular control (Full lower level API)

- New flexibilities: full low-level API, internal operations are accessible now (tf.raw_ops), and inheritable interfaces for variables, checkpoints, and layers

![alt text](https://cdn-images-1.medium.com/max/1600/0*fJ5u2WE51Oz44dr_)

## Feature #5 - Backwards compatible with Tensorflow 1.0 

- A simple script converts TF 1.0 code to TF 2.0 code 

In [0]:
!tf_upgrade_v2 --infile text_generation.py text_generation_upgraded.py

usage: tf_upgrade_v2 [-h] [--infile INPUT_FILE] [--outfile OUTPUT_FILE]
                     [--intree INPUT_TREE] [--outtree OUTPUT_TREE]
                     [--copyotherfiles COPY_OTHER_FILES] [--inplace]
                     [--reportfile REPORT_FILENAME]
tf_upgrade_v2: error: unrecognized arguments: text_generation_upgraded.py


## Feature #6 - Tensorboard is now available in Colab

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-04-05 23:00:35--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.232.181.106, 52.204.188.97, 34.226.180.131, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.232.181.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.28M  35.3MB/s    in 0.4s    

2019-04-05 23:00:35 (35.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

#HOLY SH** awesome

TODO saturday 
8-9 Stock prediction model refined https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/keras/basic_regression.ipynb 
9-10 TF Serving Explained 
10-11 Modify Old App to use New Logo, add dropdown for industry, add 2 stock visualizations + balance view
11-12 Create technical story (Demo Simple TF Serving, review its compononents, add user + payments + viz + balance + deploy functionality)
12-1:30 Record
1:30+ Meet Max
  
 Sunday 

10-11 Stock Prediction
11-12 TF Serving Explained
12-1 Modify Old App
1-2 Create Technical Story
2-3 Record
3-3:30 Chill
  

http://f1e87aa8.ngrok.io


## 4 - Build a stock prediction model w/ Tensorflow 2.0

https://colab.research.google.com/drive/1XBP0Zh8K4g_n0A2p1UlGFf3dij0EX_Kt 

## 5 - Tensorflow Serving Explained

![alt text](https://cdn-images-1.medium.com/max/1600/0*-LMpEp7g_s9_plHF.jpg)

- TensorFlow Serving is a flexible, high-performance serving system for machine learning models, designed for production environments.
- TensorFlow Serving makes it easy to deploy new algorithms and experiments, while keeping the same server architecture and APIs.
- TensorFlow Serving provides out of the box integration with TensorFlow models, but can be easily extended to serve other types of models.

## Wait, why use Serving instead of a regular web server framework like Flask or Django?


- TensorFlow-Serving allows developers to integrate client requests and data with deep learning models served independently of client systems. 
- Benefits of this include clients being able to make inferences on data without actually having to install TensorFlow or even have any contact with the actual model, and the ability to serve multiple clients with one instance of a model.

## OK, yes w could wrap a simple model in an API endpoint written in a Python framework like Flask, Falcon or similar, and voilá we have an API. But there are some really good reasons we don’t want to do it that way:


## Reason #1 - TF Serving is faster

- If your model(s) are complex and run slowly on CPU, you would want to run your models on more accelerated hardware (like GPUs). Your API-microservice(s), on the other hand, usually run fine on CPU and they’re often running in “everything agnostic” Docker containers. In that case you may want to keep those two kinds of services on different hardware.

## Reason #2 - TF Serving is more space efficient
- If you start messing up your neat Docker images with heavy TensorFlow models, they grow in every possible direction (CPU usage, memory usage, container image size, and so on). You don’t want that.

## Reason #3 - Its going a version control system built in
- Let's say your service uses multiple models written in different versions of TensorFlow. Using all those TensorFlow versions in your Python API at the same time is going to be a total mess.

- You could of course wrap one model into one API. Then you would have one service per model and you can run different services on different hardware. Perfect! Except, this is what TensorFlow Serving ModelServer is doing for you. So don’t go wrap an API around your Python code (where you’ve probably imported the entire tf library, tf.contrib, opencv, pandas, numpy, …). TensorFlow Serving ModelServer does that for you.

## More importantly, the TensorFlow team wrote TensorFlow Serving and the ModelServer for a reason. They are probably better than you when writing a high performance serving system. Use it!

## Deploy a new version of your model and let tensorflow serving gracefully finish current requests while starting to serve new requests with the new model.

## This separates concerns, data scientists can focus on building great models while Ops can focus on building highly resilient and scalable architectures that can serve those models.

![alt text](https://www.tensorflow.org/tfx/serving/images/serving_architecture.svg)

## 9 Concepts to understand in Tensorflow Serving

## Concept 1 - Servables

- Servables are the central abstraction in TensorFlow Serving. 
- Servables are the underlying objects that clients use to perform computation (for example, a lookup or inference).
- A single Servable might include anything from a single shard of a lookup table to a single model to a tuple of inference models. 

## Concept 2 - Servable Versions

- TensorFlow Serving can handle one or more versions of a servable over the lifetime of a single server instance. 
- This enables fresh algorithm configurations, weights, and other data to be loaded over time. 

## Concept 3 - Servable Streams 

- A servable stream is the sequence of versions of a servable, sorted by increasing version numbers.

## Concept 4 - Models

- TensorFlow Serving represents a model as one or more servables. 
- A machine-learned model may include one or more algorithms (including learned weights) and lookup or embedding tables.

## Concept 5 - Loaders

- Loaders manage a servable's life cycle. 

## Concept 6 - Sources

- Sources are plugin modules that find and provide servables.
-Each Source provides zero or more servable streams. 
- For each servable stream, a Source supplies one Loader instance for each version it makes available to be loaded. 

## Concept 7 - Aspired Versions

- Aspired versions represent the set of servable versions that should be loaded and ready. 
- Sources communicate this set of servable versions for a single servable stream at a time. 

## Concept 8 - Managers

- Managers handle the full lifecycle of Servables, including:

loading Servables
serving Servables
unloading Servables

## Concept 9 - Core

- Using the standard TensorFlow Serving APis, TensorFlow Serving Core manages the following aspects of servables:

lifecycle
metrics

## So the steps are

1. We train a tensroflow model called transformer on some data and are using it for inference. Then we train it on newer data.
2. The source plugin creates a loader for a specific version of this model. This loader has all the metadata necessary to load this servable computation graph, it points to it on disk.
3. The source then notifies the dynamic manager of the aspired version (the newer model)
4. The dynamic manager applies whatever version policy we write and decides to load the new version
5. The dynamic manager tells the loader there isn't enough  memory! So the loader instantiates the TF graph with the new weights. 
6. A client requests the latest version of the model, the manager returns a handle to the new version of the servable.



## BTW - Serving isn't an HTTP server, its a gRPC server 


- In gRPC a client application can directly call methods on a server application on a different machine as if it was a local object, making it easier for you to create distributed applications and services. 
- As in many RPC systems, gRPC is based around the idea of defining a service, specifying the methods that can be called remotely with their parameters and return types.
- The server side code is automatically remotely available in the client. 
- The gRPC client side has a stub that provides all the methods the server has. Of course, when the data travel over the wire, it’s no more magic than HTTP going on. But it’s the way you as a developer write the communication that is different!

My previous company Twilio can help 

[![IMAGE ALT TEXT HERE](https://img.youtube.com/vi/RoXT_Rkg8LA/0.jpg)](https://www.youtube.com/watch?v=RoXT_Rkg8LA)

## BTW - oh they recently added HTTP support, cool 

https://www.tensorflow.org/tfx/serving/api_rest

# Step 6 - TF Serving Demo 

https://github.com/tobegit3hub/simple_tensorflow_serving

## Steps 7- 9

Using https://github.com/tobegit3hub/simple_tensorflow_serving as a base we'll 

- Add User signup/login functionality
- Add payments functionality
- Deploying the app to Heroku

## Step 10 Ways of Improving the App

- Ensemble our models
- Better Design
- Learn from user interaction 
- Sentiment Analysis for news
- Try different models